In [1]:
from os import listdir
import numpy as np
from collections import defaultdict
import json
import pandas as pd
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords
import pickle


dfTrain = pd.read_pickle("dfTrainPickle.pickle")
dfValidation = pd.read_pickle("dfValidationPickle.pickle")
dfTest = pd.read_pickle("dfTestPickle.pickle")

C:\Users\Divya\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

p_stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')
en_stop = stopwords.words('english')
texts = []

for idx, t in dfTrain.iterrows():
    i = t['Content']
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    tokens = [x for x in tokens if not x.isnumeric()]
    tokens = [x for x in tokens if len(x) > 1]
    stopped_tokens = [x for x in tokens if not x in en_stop]
    stemmed_tokens = [lemmatizer.lemmatize(i) for i in stopped_tokens]
    final_tokens = stemmed_tokens
    
    # add tokens to list
    texts.append(final_tokens)
print("Done")

Done


In [3]:
pickle.dump(texts, open("cleanedLemmatizedText.p", "wb"))

In [4]:
dictionary = corpora.Dictionary(texts)
print("Done")

Done


In [5]:
dictionary.filter_extremes(no_below=20)
dictionary.compactify()
len(dictionary)

18978

In [6]:
# Save the dictionary
dictionary.save('dictionary_lemmatizer_no_below20.dict')

In [7]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [8]:
# Save the corpus
gensim.corpora.MmCorpus.serialize('BOWUnigramCorpus.mm', corpus)

In [9]:
# import logging
# logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

#ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=15, id2word = dictionary, eval_every=0, passes=5)
ldamodel = gensim.models.ldamodel.LdaModel.load('ldamodel_15topics_nobelow_20.model', mmap=None)
print("Done")
# ldamodel = gensim.models.ldamodel.LdaModel.LdaMulticore()

Done


In [17]:
# Save the model
ldamodel.save("ldamodel_15topics_nobelow_20.model")

In [10]:
ldamodel.print_topics(num_topics=15, num_words=10)

[(0,
  '0.030*"vega" + 0.022*"casino" + 0.020*"strip" + 0.014*"tower" + 0.014*"la" + 0.013*"stay" + 0.011*"pool" + 0.009*"time" + 0.009*"stayed" + 0.009*"view"'),
 (1,
  '0.030*"resort" + 0.030*"beach" + 0.020*"food" + 0.013*"day" + 0.013*"restaurant" + 0.012*"good" + 0.011*"pool" + 0.009*"water" + 0.009*"time" + 0.008*"buffet"'),
 (2,
  '0.023*"u" + 0.017*"told" + 0.016*"would" + 0.015*"desk" + 0.013*"front" + 0.013*"day" + 0.011*"check" + 0.011*"night" + 0.010*"asked" + 0.010*"one"'),
 (3,
  '0.031*"staff" + 0.021*"stay" + 0.015*"service" + 0.012*"experience" + 0.010*"u" + 0.010*"year" + 0.009*"time" + 0.009*"stayed" + 0.008*"guest" + 0.007*"trip"'),
 (4,
  '0.029*"food" + 0.028*"service" + 0.028*"bar" + 0.024*"breakfast" + 0.023*"restaurant" + 0.017*"drink" + 0.010*"dinner" + 0.010*"coffee" + 0.008*"table" + 0.008*"great"'),
 (5,
  '0.040*"parking" + 0.028*"car" + 0.016*"airport" + 0.014*"park" + 0.014*"shuttle" + 0.014*"street" + 0.012*"san" + 0.012*"walk" + 0.010*"valet" + 0.010*"

In [11]:
dfTrain[['Overall', 'Cleanliness', 'Location', 'Rooms', 'Service', 'Value']] = dfTrain[['Overall', 'Cleanliness', 'Location', 'Rooms', 'Service', 'Value']].apply(pd.to_numeric)
dfValidation[['Overall', 'Cleanliness', 'Location', 'Rooms', 'Service', 'Value']] = dfValidation[['Overall', 'Cleanliness', 'Location', 'Rooms', 'Service', 'Value']].apply(pd.to_numeric)
dfTest[['Overall', 'Cleanliness', 'Location', 'Rooms', 'Service', 'Value']] = dfTest[['Overall', 'Cleanliness', 'Location', 'Rooms', 'Service', 'Value']].apply(pd.to_numeric)

In [13]:
print(dfTrain['Content'][4])
ldamodel.get_document_topics(corpus[4])

We went for a girls long weekend trip. The hotel is very nice, the rooms extra clean and comfortable. The pool area is beautiful. We did notice this is a very popular hotel for couples, there were a few families while we were there, but not that many children. All of the hotel staff was very friendly, and went above and beyond to accommodate anything we needed. It was such a nice, relaxing weekend. Some pluses: drinks by the pool (on the pricey side, but were good), service to the beach chairs out on the beach, and walking distance to several good restaurants and nightclubs. Also, you are an easy taxi ride to spend a day in Old San Juan ($19 each way), and about an hour to El Yunque rainforest. I highly recommend renting a car for a day and making that trip. Swimming in the waterfall is an amazing (albeit cold!) experience. You can rent a car at the hotel from Avis. You will save money if you go online and rent vs doing it at the counter. Also keep in mind if you rent a car and keep it

[(0, 0.04569156278811632),
 (1, 0.19002886392193086),
 (2, 0.035362430535593362),
 (4, 0.040562706423538569),
 (5, 0.22941681255187216),
 (8, 0.10238108273498908),
 (13, 0.2821586358265129),
 (14, 0.070991571642872509)]

In [14]:
#Regression

def cleanBOWModel(text):
    raw = text.lower()
    tokens = tokenizer.tokenize(raw)
    tokens = [x for x in tokens if not x.isnumeric()]
    tokens = [x for x in tokens if len(x) > 1]
    stopped_tokens = [x for x in tokens if not x in en_stop]
    stemmed_tokens = [lemmatizer.lemmatize(i) for i in stopped_tokens]
    return dictionary.doc2bow(stemmed_tokens)

def feature(datum, ldaModel):
    feat = [0]*(1+len(ldaModel.get_topics()))
    feat[0] = 1
    i = cleanBOWModel(datum['Content'])
    review_topic_model = ldaModel[i]
    for x in review_topic_model:
        feat[x[0]+1] = x[1]
    return feat

def featureTrain(datum, ldaModel):
    feat = [0]*(1+len(ldaModel.get_topics()))
    feat[0] = 1
    review_topic_model = ldaModel[datum]
    for x in review_topic_model:
        feat[x[0]+1] = x[1]
    return feat

# Service
X_train = [featureTrain(d, ldamodel) for d in corpus]
y_train_service = dfTrain['Service'].tolist()
y_train_cleanliness = dfTrain['Cleanliness'].tolist()
y_train_location = dfTrain['Location'].tolist()
y_train_rooms = dfTrain['Rooms'].tolist()
y_train_value = dfTrain['Value'].tolist()
y_train_overall = dfTrain['Overall'].tolist()

print("Built training set")

X_validation = [feature(d, ldamodel) for idx, d in dfValidation.iterrows()]
X_test = [feature(d, ldamodel) for idx, d in dfTest.iterrows()]

print("Built test/validation set")
y_validation = []
y_test = []

for aspect in ['Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall']:
    y_curr = dfValidation[aspect].tolist()
    y_validation.append(y_curr)
    y_curr = dfTest[aspect].tolist()
    y_test.append(y_curr)

# y_validation_service = [d['Service'] for idx, d in dfValidation.iterrows()]
# y_validation_cleanliness = [d['Cleanliness'] for idx, d in dfValidation.iterrows()]
# y_validation_location = [d['Location'] for idx, d in dfValidation.iterrows()]
# y_validation_rooms = [d['Rooms'] for idx, d in dfValidation.iterrows()]
# y_validation_value = [d['Value'] for idx, d in dfValidation.iterrows()]
# y_validation_overall = [d['Overall'] for idx, d in dfValidation.iterrows()]

# y_test_service = [d['Service'] for idx, d in dfTest.iterrows()]
# y_test_cleanliness = [d['Cleanliness'] for idx, d in dfTest.iterrows()]
# y_test_location = [d['Location'] for idx, d in dfTest.iterrows()]
# y_test_rooms = [d['Rooms'] for idx, d in dfTest.iterrows()]
# y_test_value = [d['Value'] for idx, d in dfTest.iterrows()]
# y_test_overall = [d['Overall'] for idx, d in dfTest.iterrows()]

Built training set
Built test/validation set


In [33]:
print(dfTest['Content'][7])
print(ldamodel[cleanBOWModel(dfTest['Content'][7])])
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

Stayed here in August 2005, bit of a delay in writing review!!! Location: Good location, just around the corner from various plazas, 10 minutes walk from Puerto Del Sol. Hotel: receptionist friendly and helpful. Hotel can appear a bit dark inside due to decor, but v clean and modern. Room: Quite small room, and appears dingy at times due to decor. But spotless and enough space and storage for 2 people. Bathroom is stunning - perfect for girls geting ready to go out. Big mirrors and good lighting. Power shower, big bath and 2 bowl sinks. Bathroom door is frosted glass so limited privacy at times. Bed is big but firm. Air con worked a treat. Didnt spend much time in hotel room anyway - there's too much to see. Sight-seeing: Get tickets for the tour buses so you can hop-on and off whenever you like. You'll get to see al the attractions and find things you want to see inc various plazas, stadium etc. Madrid is wonderful, enjoy it!
[(1, 0.052463388091768488), (3, 0.036873153793158579), (5, 

In [36]:
y_train = []
y_train.append(y_train_service)
y_train.append(y_train_cleanliness)
y_train.append(y_train_location)
y_train.append(y_train_rooms)
y_train.append(y_train_value)
y_train.append(y_train_overall)
print(len(y_train))
print(len(y_train[0]))

for idx, aspect in enumerate(['Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall']):
    print("\nAspect: ", aspect, "\n")
    clf = linear_model.SGDRegressor(tol=1e-6, max_iter=1e+6)
    clf.fit(X_train, y_train[idx])
    y_pred_train = clf.predict(X_train)
    y_pred_validation = clf.predict(X_validation)
    y_pred_test = clf.predict(X_test)
    print(mean_squared_error(y_train[idx], y_pred_train))
    print(mean_squared_error(y_validation[idx], y_pred_validation))
    print(mean_squared_error(y_test[idx], y_pred_test))
    print("True service rating: "+str(y_test[idx][7]))
    print("Predicted rating: "+str(y_pred_test[7]))
    print("Content of review: "+dfTest['Content'][7])
    print(clf.coef_)

6
280000

Aspect:  Service 

1.08665034022
1.1294441574
1.09285031519
True service rating: 4
Predicted rating: 3.64670182568
Content of review: Stayed here in August 2005, bit of a delay in writing review!!! Location: Good location, just around the corner from various plazas, 10 minutes walk from Puerto Del Sol. Hotel: receptionist friendly and helpful. Hotel can appear a bit dark inside due to decor, but v clean and modern. Room: Quite small room, and appears dingy at times due to decor. But spotless and enough space and storage for 2 people. Bathroom is stunning - perfect for girls geting ready to go out. Big mirrors and good lighting. Power shower, big bath and 2 bowl sinks. Bathroom door is frosted glass so limited privacy at times. Bed is big but firm. Air con worked a treat. Didnt spend much time in hotel room anyway - there's too much to see. Sight-seeing: Get tickets for the tour buses so you can hop-on and off whenever you like. You'll get to see al the attractions and find th

In [50]:
dfAggregatedTrain = dfTrain.groupby('HotelId')['HotelId', 'Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall'].agg(np.mean)
# dfAggregatedValidation = dfValidation.groupby('HotelId')['HotelId', 'Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall'].agg(np.mean)
dfAggregatedTrain.head()
# dfAggregatedTest = dfValidation.groupby('HotelId')['HotelId', 'Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall'].agg(np.mean)

,HotelId,Service,Cleanliness,Location,Rooms,Value,Overall
HotelId,,,,,,,
0,0.0,4.428571,4.428571,4.428571,4.428571,4.571429,4.285714
1,1.0,4.529412,4.720588,4.647059,4.382353,4.088235,4.397059
2,2.0,3.416667,3.437500,4.145833,3.354167,3.270833,3.125000
3,3.0,1.750000,1.500000,3.833333,1.166667,1.250000,1.166667
4,4.0,4.779221,4.857143,4.909091,4.584416,4.389610,4.649351


In [72]:
# Baseline

for id, aspect in enumerate(['Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall']):
    print("\nAspect: ", aspect, "\n")
    y_pred_train1 = []
    for idx, r in dfTrain.iterrows():
        currHotelID = r['HotelId']
        #print(dfAggregatedTrain.loc[dfAggregatedTrain['HotelId'] == currHotelID][aspect].item())
        y_pred_train1.append(dfAggregatedTrain.loc[dfAggregatedTrain['HotelId'] == currHotelID][aspect].item())
        #print(y_pred_train1)
        #break
    #y_pred_train = clf.predict(X_train)
    #y_pred_validation = clf.predict(X_validation)
    #y_pred_test = clf.predict(X_test)
    print(mean_squared_error(y_train[id], y_pred_train1))
    #print(mean_squared_error(y_validation[idx], y_pred_validation))
    #print(mean_squared_error(y_test[idx], y_pred_test))


Aspect:  Service 

1.54600120881

Aspect:  Cleanliness 

1.17247201725

Aspect:  Location 

0.829872963555

Aspect:  Rooms 

1.31181734718

Aspect:  Value 

1.56943674947

Aspect:  Overall 

1.45333195196


In [73]:
#Baseline 2

for id, aspect in enumerate(['Service', 'Cleanliness', 'Location', 'Rooms', 'Value']):
    print("\nAspect: ", aspect, "\n")
    y_pred_train1 = dfTrain['Overall'].tolist()
    y_pred_validation1 = dfValidation['Overall'].tolist()
    y_pred_test1 = dfTest['Overall'].tolist()
    #y_pred_train = clf.predict(X_train)
    #y_pred_validation = clf.predict(X_validation)
    #y_pred_test = clf.predict(X_test)
    print(mean_squared_error(y_train[id], y_pred_train1))
    print(mean_squared_error(y_validation[id], y_pred_validation1))
    print(mean_squared_error(y_test[id], y_pred_test1))


Aspect:  Service 

0.645817857143
0.680708372629
0.65638

Aspect:  Cleanliness 

0.875464285714
0.970279262286
0.90485

Aspect:  Location 

1.89798571429
2.15139369171
1.95156

Aspect:  Rooms 

0.588525
0.631942261343
0.60451

Aspect:  Value 

0.557032142857
0.564851199832
0.56783


In [95]:
c1 = 100
for x in X_validation[100:125]:
    c2 = 1000
    for y in X_test[1000:]:
        if(np.dot(x[1:], y[1:]) == 0 and len(dfValidation['Content'][c1]) > 30 and len(dfTest['Content'][c2]) > 30):
            print(dfValidation['Content'][c1])
            print(ldamodel[cleanBOWModel(dfValidation['Content'][c1])])
            print(dfTest['Content'][c2])
            print(ldamodel[cleanBOWModel(dfTest['Content'][c2])])
            print("--------------------------------------------------------------------------------------------------------")
            break
        c2 += 1
    c1 += 1
print("Done")

I have been to Vegas many times and have walked thru the Aria lobby/casino but this was the first stay there as we were attending the Cirque Viva Elvis show and wanted to stay close by. Let's start with the basic pluses; newer hotel so rooms are in great shape; beds are very comfortable and you can get a good nights sleep there (once you make it to your room)!; everything in your room run by your remote which is great when the batteries are working (had to have maintenance come and change those when we checked in as nothing worked with the remote); Minuses: buffet is good value but selection is not as good as the Bellagio (you must eat at Bellagio one time as it is incredible for a buffet!); food at the Jean Phillipe cafe in the lobby and little cafe on the second floor is really expensive for just a sandwich with just OK panini sandwiches; hotel has a strong "waffle cone" smell when you walk in the door (most likely from the Jean Phillipe place in the lobby) and does not go away - it 

--------------------------------------------------------------------------------------------------------
Stayed here in October,2011, for 13 days, and couldn't be happier with accomodations, and especially the helpful staff. Located about 1 km north of the main railroad station, and about 1-l/2 km east of the Borghese Gardens, this villa (built in 1900) was about 100 meters from a bus and tram stop, and a 5-to-10 minute walk to Metro stop. In a quiet, tree-lined neighborhood within walking distance of really good, economical restaurants, perfect after a long day's touring, or even a hectic day of business. Rooms called small by other reviewers were spacious by European standards, my pal and I moved around easily. Terraces were available to all, complete with chaises-longues, for relaxing in the mild Roman nights. The pricing for rooms was low compared to the dubious privilege of staying in the Centro Storico, or anywhere near it, any why bother when you are 10 to 15 minutes by taxi or 

In [97]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(vis_data)

C:\Users\Divya\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [114]:
import pickle
with open('dfTrain_HighestProtocol', 'wb') as output:
    pickle.dump(dfTrain, output, 2)
with open('dfTest_HighestProtocol', 'wb') as output:
    pickle.dump(dfTest, output, 2)
with open('dfTValidation_HighestProtocol', 'wb') as output:
    pickle.dump(dfValidation, output, 2)

In [126]:
#Model with prior
idToTopic = {0:'Value', 1:'Room', 2:'Location', 3:'Cleanliness', 4:'Service', 5:'Checkin', 6:'BusinessService', 7:'Positive', 8:'Negative'}
numberOfTopics = len(idToTopic)

priorMapping = defaultdict(list)
priorMapping['Value'] = ['value', 'price', 'quality', 'worth', 'expensive', 'cheap', 'money', 'charged']
priorMapping['Room'] = ['bedroom', 'suite', 'view', 'bed', 'bathroom', ]
priorMapping['Location'] = ['location', 'traffic', 'minute', 'restaurant', 'airport', 'shuttle', 'taxi', 'bus']
priorMapping['Checkin'] = ['checkin', 'desk', 'front', 'booked', 'reservation', 'manager']
priorMapping['BusinessService'] = ['internet', 'business', 'computer', 'center']
priorMapping['Cleanliness'] = ['clean', 'dirty', 'maintain', 'smell']
priorMapping['Service'] = ['service', 'help', 'welcome', 'breakfast', 'buffet', 'comfortable']
priorMapping['Positive'] = ['great', 'amazing', 'recommend', 'wonderful', 'friendly', 'helpful', 'perfect', 'excellent']
priorMapping['Negative'] = ['rude', 'smell', 'noise', 'loud', 'apology', 'charged', 'poor', 'cancel']

In [130]:
dictionaryPrior = corpora.Dictionary(texts[:50000])
dictionaryPrior.filter_extremes(no_below=20)
dictionaryPrior.compactify()
corpusPrior = [dictionaryPrior.doc2bow(text) for text in texts[:50000]]

In [141]:
priorDistribution = np.zeros(shape=(numberOfTopics, len(dictionary)))
priorDistribution.fill(0.000001)
for i in range(numberOfTopics):
    for word in priorMapping[idToTopic[i]]:
        if(dictionary.token2id[word]>=0):
            priorDistribution[i][dictionary.token2id[word]] = 1

In [144]:
import logging
import time
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)

%time ldamodelPrior = gensim.models.ldamulticore.LdaMulticore(corpusPrior, num_topics=9, id2word = dictionary, eval_every=1, passes=20, iterations=50, eta=priorDistribution,\
                                                      alpha='asymmetric')

C:\Users\Divya\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:728: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Wall time: 23min 9s


In [145]:
vis_data = gensimvis.prepare(ldamodelPrior, corpusPrior, dictionaryPrior, sort_topics=False)
pyLDAvis.display(vis_data)

C:\Users\Divya\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [175]:
#Random Forest Classifier

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

for idx, aspect in enumerate(['Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall']):
    print("\nAspect: ", aspect, "\n")
    clf = RandomForestClassifier(n_estimators=20, max_features=4, min_samples_split=2, random_state=0)
    clf.fit(X_train, y_train[idx])
    y_pred_train = clf.predict(X_train)
    #y_pred_validation = clf.predict(X_validation)
    y_pred_test = clf.predict(X_test)
    print("Precision:")
    print(metrics.precision_score(y_train[idx], y_pred_train, pos_label=1, average='macro'))
    print(metrics.precision_score(y_test[idx], y_pred_test, pos_label=1, average='macro'))
    print("Recall:")
    print(metrics.recall_score(y_train[idx], y_pred_train, pos_label=1, average='macro'))
    print(metrics.recall_score(y_test[idx], y_pred_test, pos_label=1, average='macro'))
    print("F1:")
    print(metrics.f1_score(y_train[idx], y_pred_train, pos_label=1, average='macro'))
    print(metrics.f1_score(y_test[idx], y_pred_test, pos_label=1, average='macro'))
    print("Accuracy:")
    print(metrics.accuracy_score(y_train[idx], y_pred_train))
    print(metrics.accuracy_score(y_test[idx], y_pred_test))


Aspect:  Service 

Precision:
0.99654660223
0.560408001852
Recall:
0.994418228136
0.53721376856
F1:
0.995469828206
0.540479579337
Accuracy:
0.995146428571
0.60103

Aspect:  Cleanliness 

Precision:
0.996797111845
0.556440883914
Recall:
0.994889407727
0.534781784657
F1:
0.995830328211
0.537684106335
Accuracy:
0.995346428571
0.6161

Aspect:  Location 

Precision:
0.997826678485
0.568486487243
Recall:
0.994716623198
0.442545691817
F1:
0.996262875432
0.481545870006
Accuracy:
0.995664285714
0.61162

Aspect:  Rooms 

Precision:
0.995991209254
0.542472875972
Recall:
0.994706605984
0.5380037722
F1:
0.995335981993
0.536123676191
Accuracy:
0.994696428571
0.55628

Aspect:  Value 

Precision:
0.996150295742
0.530031074505
Recall:
0.994809319698
0.521324012833
F1:
0.995467799685
0.518681162605
Accuracy:
0.994917857143
0.54707

Aspect:  Overall 

Precision:
0.99594001408
0.57307444179
Recall:
0.99457197591
0.561085524207
F1:
0.995244034269
0.560345160379
Accuracy:
0.994746428571
0.59955


In [176]:
#Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

for idx, aspect in enumerate(['Service', 'Cleanliness', 'Location', 'Rooms', 'Value', 'Overall']):
    print("\nAspect: ", aspect, "\n")
    clf = RandomForestRegressor(max_depth=2, random_state=0)
    clf.fit(X_train, y_train[idx])
    y_pred_train = clf.predict(X_train)
    y_pred_test = clf.predict(X_test)
    print("MSE:")
    print(mean_squared_error(y_train[idx], y_pred_train))
    print(mean_squared_error(y_test[idx], y_pred_test))


Aspect:  Service 

MSE:
1.31778496705
1.33568656139

Aspect:  Cleanliness 

MSE:
1.06290664601
1.08433294657

Aspect:  Location 

MSE:
0.981648260733
1.00468729168

Aspect:  Rooms 

MSE:
1.18196179917
1.18492769998

Aspect:  Value 

MSE:
1.27503073978
1.28447041577

Aspect:  Overall 

MSE:
1.12755895133
1.13351214279


1.33568656139
